In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

34123


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

# Update H2B.S

buttress_histone_2022

## Update description

In [18]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H2B.S_(Magnoliopsida)'"
)
cursor.execute(query)
df_h2bs = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df_h2bs

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H2B.S_(Magnoliopsida),variant_group,Magnoliopsida,3398,40,H2B,40,H2B.S_(Magnoliopsida) is a class a new class ...,Flowering plants (angiosperms) [jiang_evolutio...,"HTB8 gene in Arabidopsis, Solyc06g074750.1 in ...",...,null,For Arabidopsis an adaptive function in cell t...,The angiosperm-specific clade of Arabidopsis H...,null,null,"Two HTB8 residues, Arg152 and Met179, were hig...",null,null,null,None


In [12]:
df_h2bs["summary"].values[0]

'H2B.S_(Magnoliopsida)  is a class a new class of highly divergent H2B variants identified by Jiang et al. that specifically accumulate during chromatin compaction of dry seed embryos in multiple species of flowering plants [jiang_evolution_2020].'

In [19]:
description = "H2B.S_(Magnoliopsida) is a class a new class of highly divergent H2B variants identified by Jiang et al. that specifically accumulate during chromatin compaction of dry seed embryos in multiple species of flowering plants [jiang_evolution_2020, buttress_histone_2022]."
function = "The histone variant H2B.S has been shown to condense chromatin through its intrinsically disordered region (IDR) by selectively aggregating transcriptionally inactive AT-rich regions via phase separation, while excluding active genes to maintain transcriptional competence [buttress_histone_2022]."
query = f"UPDATE histone_description SET summary='{description}', function='{function}' WHERE id=40"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary='H2B.S_(Magnoliopsida) is a class a new class of highly divergent H2B variants identified by Jiang et al. that specifically accumulate during chromatin compaction of dry seed embryos in multiple species of flowering plants [jiang_evolution_2020, buttress_histone_2022].', function='The histone variant H2B.S has been shown to condense chromatin through its intrinsically disordered region (IDR) by selectively aggregating transcriptionally inactive AT-rich regions via phase separation, while excluding active genes to maintain transcriptional competence [buttress_histone_2022].' WHERE id=40


In [20]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H2B.S_(Magnoliopsida)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H2B.S_(Magnoliopsida),variant_group,Magnoliopsida,3398,40,H2B,40,H2B.S_(Magnoliopsida) is a class a new class o...,Flowering plants (angiosperms) [jiang_evolutio...,"HTB8 gene in Arabidopsis, Solyc06g074750.1 in ...",...,null,The histone variant H2B.S has been shown to co...,The angiosperm-specific clade of Arabidopsis H...,null,null,"Two HTB8 residues, Arg152 and Met179, were hig...",null,null,null,None


## Add histone publication

In [21]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_has_publication hp "
    "ON h.id = hp.histone_id "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].isin(["H2B.S_(Magnoliopsida)"])]

,id,level,taxonomic_span,taxonomic_span_id,description,parent,histone_id,publication_id
335,H2B.S_(Magnoliopsida),variant_group,Magnoliopsida,3398,40.0,H2B,H2B.S_(Magnoliopsida),jiang_evolution_2020


In [22]:
pid = "buttress_histone_2022"
query = f"SELECT * FROM publication WHERE id='{pid}'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,title,doi,author,year


In [23]:
data_publication = {
    "id": pid,
    "title": "Histone H2B.8 compacts flowering plant sperm through chromatin phase separation",
    "doi": "10.1038/s41586-022-05386-6",
    "author": None,
    "year": "2022",
}
cursor.execute(add_publication, data_publication)

In [24]:
cursor.execute(add_histone_has_publication, ("H2B.S_(Magnoliopsida)", pid))

In [25]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_has_publication hp "
    "ON h.id = hp.histone_id "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].isin(["H2B.S_(Magnoliopsida)"])]

,id,level,taxonomic_span,taxonomic_span_id,description,parent,histone_id,publication_id
335,H2B.S_(Magnoliopsida),variant_group,Magnoliopsida,3398,40.0,H2B,H2B.S_(Magnoliopsida),buttress_histone_2022
336,H2B.S_(Magnoliopsida),variant_group,Magnoliopsida,3398,40.0,H2B,H2B.S_(Magnoliopsida),jiang_evolution_2020


In [26]:
# Make sure data is committed to the database
conn.commit()

## Add alternate name

In [31]:
query = (
    "SELECT * FROM histone h LEFT JOIN alternative_name ha "
    "ON h.id = ha.histone "
    "WHERE h.id = 'H2B.S_(Magnoliopsida)' "
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,name,taxonomy,gene,splice,histone
0,H2B.S_(Magnoliopsida),variant_group,Magnoliopsida,3398,40,H2B,None,None,None,None,None,None


In [32]:
data_alternate_name = {
    "name": "H2B.8",
    "taxonomy": None,
    "gene": None,
    "splice": None,
    "histone": "H2B.S_(Magnoliopsida)",
}
cursor.execute(add_alternate_names, data_alternate_name)

In [33]:
query = (
    "SELECT * FROM histone h LEFT JOIN alternative_name ha "
    "ON h.id = ha.histone "
    "WHERE h.id = 'H2B.S_(Magnoliopsida)' "
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,name,taxonomy,gene,splice,histone
0,H2B.S_(Magnoliopsida),variant_group,Magnoliopsida,3398,40,H2B,90,H2B.8,None,None,None,H2B.S_(Magnoliopsida)


In [34]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [35]:
cursor.close()
conn.close()
tunnel.stop()